# Generation of Habitat and Landcover Layers and Resistance Dictionaries

This notebook generates the habitat layer, landcover layer, and resistance dictionary for each specified bird species based on data from the IUCN Red List and eBird, which are all needed as inputs to the main model. It uses the `ecoscape_layers` package; more information about usage and parameters can be found at https://github.com/ecoscape-earth/ecoscape-layers.

At minimum, you will need:
- API keys for:
    - [the IUCN Red List](http://apiv3.iucnredlist.org/)
    - [eBird](https://science.ebird.org/en/status-and-trends/download-data) (no need for the R package, just request an access key)
- The 6-letter eBird code(s) for the bird species you are creating layers for. You can obtain these codes by looking up bird species on eBird and taking the last 6 letters of the species page's URL (such as "acowoo" from https://ebird.org/species/acowoo).
- An initial landcover map. For EcoScape, we derive our map from the map `iucn_habitatclassification_composite_lvl2_ver004.zip` (cropped to the United States, which is our broad area of study) provided by [Jung et al.](https://zenodo.org/record/4058819).

In [ ]:
import os
from ecoscape_layers import LayerGenerator

In [ ]:
DATA_PATH = ".."

The given parameters specify the parameters used for the EcoScape paper results. You can change them as you see fit.

In [2]:
# REQUIRED INPUTS

REDLIST_KEY = input("IUCN Red List API key: ")
EBIRD_KEY = input("EBird API key: ")

species_list = ["acowoo", "stejay"]
landcover_path = os.path.join(DATA_PATH, "inputs", "iucn_habclass_lvl2_us.tif")

# OPTIONAL INPUTS

crs = 'PROJCS["Albers_Conical_Equal_Area",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'
resolution = 300
resampling = "near"
bounds = (-2361583.2255652365274727, 1242360.4432584622409195, -1646660.4594384517986327, 2455525.7292816736735404)
padding = 200000

In [ ]:
# Initialize LayerGenerator instance.
layer_generator = LayerGenerator(REDLIST_KEY, EBIRD_KEY, landcover_path, crs, resolution, resampling, bounds, padding)

In [ ]:
# Process the landcover first, which is shared between the bird species.
layer_generator.process_landcover()

# Then run the habitat layer generation process for each bird species.
for species_code in species_list:
    habitat_fn = os.path.join(DATA_PATH, "outputs", species_code, "habitat.tif")
    resistance_dict_fn = os.path.join(DATA_PATH, "outputs", species_code, "resistance.csv")
    range_fn = os.path.join(DATA_PATH, "outputs", species_code, "range_map_2020.gpkg")

    if species_code == "acowoo":
        refine_method = "forest_add308"
    else:
        refine_method = "forest"

    layer_generator.generate_habitat(species_code, habitat_fn, resistance_dict_fn, range_fn, refine_method)